In [1]:
import pandas as pd
import numpy as np

In [2]:
cols= ['account_id       ','oper_date       ', 'CP       ', 'resulted_rate       ', 'resulted_amount       ', \
       'resulted_weight       ','rate       ', 'amount       ', 'weight       ']

In [3]:
df = pd.read_csv("Tsup_4393490_trade_history-20220727.csv", skipinitialspace= True, usecols= cols)

In [4]:
df.columns = df.columns.str.strip() #Removing spaces in the column names

In [5]:
df['oper_date_AMPM'] = df['oper_date'].str[-2:] #Separating am/pm into separate column

In [6]:
df["oper_date"] = pd.to_datetime(pd.Series(df["oper_date"]), format='%d-%b-%y %I.%M.%S.%f %p') #Converting into datetime and 24 hrs format
df["oper_date"] = df["oper_date"].apply(lambda x: x.replace(microsecond=0)) # Removing microseconds from the string

In [7]:
df = df.sort_values(by=['account_id','CP','oper_date'], ascending=[True, True, True]) #Sorting the values
df["cashflow"] = -df["amount"] * df["rate"] # Calculating cashflow
df["year"] = df['oper_date'].astype(str).str[0:4] # Creating Year column from oper_date column

In [8]:
df

,account_id,CP,oper_date,resulted_rate,resulted_amount,resulted_weight,rate,amount,weight,oper_date_AMPM,cashflow,year
2902,598192,AUD/CAD,2014-01-06 09:33:18,0.954430,-10000.00,-8946.90,0.95443,-10000.00,-8946.90,AM,9544.30000,2014
2906,598192,AUD/CAD,2014-01-06 11:21:50,0.954430,0.00,0.00,0.95551,10000.00,8952.60,AM,-9555.10000,2014
3042,598192,AUD/CAD,2014-04-09 22:22:04,1.025490,1000.00,941.71,1.02549,1000.00,941.71,PM,-1025.49000,2014
3062,598192,AUD/CAD,2014-04-10 00:49:50,1.025490,0.00,0.00,1.02505,-1000.00,-940.73,AM,1025.05000,2014
3662,598192,AUD/CAD,2020-03-06 05:53:08,0.890670,10000.00,6640.95,0.89067,10000.00,6640.95,AM,-8906.70000,2020
...,...,...,...,...,...,...,...,...,...,...,...,...
6156685,1707031,LTC/USD,2022-06-17 10:16:34,45.942278,-0.06,-2.76,46.01200,0.01,0.46,AM,-0.46012,2022
6156686,1707031,LTC/USD,2022-06-17 10:16:34,45.942278,-0.05,-2.30,46.01200,0.01,0.46,AM,-0.46012,2022
6156687,1707031,LTC/USD,2022-06-17 10:16:57,45.942278,-0.02,-0.92,45.99200,0.01,0.46,AM,-0.45992,2022
6156688,1707031,LTC/USD,2022-06-17 10:16:57,45.942278,-0.01,-0.46,45.99200,0.01,0.46,AM,-0.45992,2022


In [9]:
unique_accid = df["account_id"].unique().tolist() #Getting the unique account id's

In [10]:
unique_accid
df = df[["account_id", "CP", "oper_date", "resulted_amount", "cashflow", "year"]].reset_index() # Filtering selected columns
df = df[["account_id", "CP", "oper_date", "resulted_amount", "cashflow", "year"]] # Removing index column

In [11]:
df.head()

,account_id,CP,oper_date,resulted_amount,cashflow,year
0,598192,AUD/CAD,2014-01-06 09:33:18,-10000.0,9544.30,2014
1,598192,AUD/CAD,2014-01-06 11:21:50,0.0,-9555.10,2014
2,598192,AUD/CAD,2014-04-09 22:22:04,1000.0,-1025.49,2014
3,598192,AUD/CAD,2014-04-10 00:49:50,0.0,1025.05,2014
4,598192,AUD/CAD,2020-03-06 05:53:08,10000.0,-8906.70,2020


In [12]:
df['invested'] = np.where(df['cashflow']<= 0, -df['cashflow'], 0)

In [13]:
full_list = []
open_trades =pd.DataFrame()
for i in unique_accid:
    temp_acc_df = df[df["account_id"] == i]
    unique_cp = temp_acc_df["CP"].unique().tolist()
    for j in unique_cp:
        temp_cp_df = temp_acc_df[temp_acc_df["CP"] ==j].reset_index()
        len_temp_cp_df = len(temp_cp_df)
        #print(len(temp_cp_df))
        ctr = 0
        for k, row in temp_cp_df.iterrows():
            finder = 0
            if temp_cp_df["resulted_amount"][k] == 0.0:
                finder = k
                temp_df_fil = temp_cp_df[ctr:finder+1]
                profit = temp_df_fil["cashflow"].sum()
                trades = len(temp_df_fil)
                invested =  temp_df_fil["invested"].sum()
                if profit>0:
                    win = 1
                    loss = 0
                else:
                    win = 0
                    loss = 1
                ctr = finder+1
                full_list.append([row["account_id"], row["CP"], row["oper_date"],row["resulted_amount"],row["cashflow"],row["year"], profit, invested, win, loss, trades])
        if temp_cp_df["resulted_amount"].iat[-1]!=0.0:
            missing = temp_cp_df[ctr:len_temp_cp_df]
            if len(missing)>0:
                open_trades = open_trades.append(missing)

In [38]:
open_trades.to_csv('open.csv', index= False)

In [15]:
open_trades.drop(columns=["index"])

,account_id,CP,oper_date,resulted_amount,cashflow,year,invested
6177,598192,EUR/USD,2022-07-26 17:11:55,10000.0,-10123.60,2022,10123.60
7866,867318,EUR/USD,2022-07-19 07:17:37,-5000.0,5132.80,2022,0.00
7867,867318,EUR/USD,2022-07-19 07:20:50,-4000.0,-1026.33,2022,1026.33
7868,867318,EUR/USD,2022-07-19 07:20:52,-3000.0,-1026.33,2022,1026.33
7869,867318,EUR/USD,2022-07-19 07:26:22,-2000.0,-1025.98,2022,1025.98
...,...,...,...,...,...,...,...
1503,1703339,USD/CAD,2022-07-26 11:06:12,-3000.0,-1288.32,2022,1288.32
1504,1703339,USD/CAD,2022-07-26 11:49:39,-4000.0,1288.97,2022,0.00
1505,1703339,USD/CAD,2022-07-26 11:49:39,-3000.0,-1289.20,2022,1289.20
1506,1703339,USD/CAD,2022-07-26 14:33:34,-4000.0,1289.86,2022,0.00


In [16]:
open_trades.to_csv('opentrades.csv', index= False)

In [17]:
cols = ["account_id", "CP", "oper_date", "resulted_amount", "cashflow", "year", "profit_loss", "invested", "win", "loss","transactions"]
fin_df = pd.DataFrame(full_list, columns= cols)

In [18]:
fin_df.to_csv("finaldf.csv", index=False)

In [19]:
fin_df

,account_id,CP,oper_date,resulted_amount,cashflow,year,profit_loss,invested,win,loss,transactions
0,598192,AUD/CAD,2014-01-06 11:21:50,0.0,-9555.10000,2014,-10.80000,9555.10000,0,1,2
1,598192,AUD/CAD,2014-04-10 00:49:50,0.0,1025.05000,2014,-0.44000,1025.49000,0,1,2
2,598192,AUD/CAD,2020-03-08 18:02:20,0.0,8979.10000,2020,72.40000,8906.70000,1,0,2
3,598192,AUD/CAD,2020-03-10 01:08:42,0.0,8934.90000,2020,-19.20000,17950.40000,0,1,4
4,598192,AUD/CAD,2020-03-11 15:13:05,0.0,8932.50000,2020,-22.20000,8954.70000,0,1,2
...,...,...,...,...,...,...,...,...,...,...,...
283961,1707031,LTC/USD,2022-06-17 09:31:00,0.0,-0.46462,2022,-0.00233,0.46462,0,1,2
283962,1707031,LTC/USD,2022-06-17 09:31:02,0.0,-0.46462,2022,-0.00233,0.46462,0,1,2
283963,1707031,LTC/USD,2022-06-17 09:31:06,0.0,-0.46471,2022,-0.00233,0.46471,0,1,2
283964,1707031,LTC/USD,2022-06-17 10:06:03,0.0,0.46109,2022,-3.85041,109.31263,0,1,461


In [20]:
final = (fin_df.groupby(['account_id', 'CP']).agg(win=('win', 'sum'), loss=('loss', 'sum'), profit_loss=("profit_loss","sum"), invested=("invested", "sum"), transactions=("transactions", "sum")).reset_index())

In [21]:
final["win_loss_ratio"] = round((final["win"] / (final["win"] + final["loss"])),2)
final["profit%"] = round((final["profit_loss"] / final["invested"]), 5)

In [22]:
final

,account_id,CP,win,loss,profit_loss,invested,transactions,win_loss_ratio,profit%
0,598192,AUD/CAD,2,6,123.350000,5.857373e+05,28,0.25,0.00021
1,598192,AUD/CHF,1,5,-19.070000,3.162175e+04,14,0.17,-0.00060
2,598192,AUD/JPY,14,33,-124005.000000,5.807308e+07,114,0.30,-0.00214
3,598192,AUD/NZD,3,4,46.490000,1.343748e+05,14,0.43,0.00035
4,598192,AUD/USD,49,70,5747.710000,8.393345e+06,369,0.41,0.00068
...,...,...,...,...,...,...,...,...,...
1616,1703339,EUR/USD,135,61,786.240000,2.294983e+06,2144,0.69,0.00034
1617,1703339,GBP/CAD,37,11,384.440000,8.159073e+05,920,0.77,0.00047
1618,1703339,GBP/JPY,95,53,442310.000000,2.305444e+08,2230,0.64,0.00192
1619,1703339,USD/CAD,81,24,831.790000,2.872437e+06,1492,0.77,0.00029


In [23]:
final.to_csv("Output.csv",index=None)

# Traders with best win/loss ratio

In [45]:
traders_win_loss= final.groupby(['account_id', 'CP']).agg({'win': 'sum', 'loss': 'sum'}).reset_index()

In [46]:
traders_win_loss['win_loss_ratio'] = traders_win_loss['win'] / traders_win_loss['loss']
traders_win_loss= traders_win_loss.sort_values(by = ['win_loss_ratio'], ascending=False)


In [48]:
traders_win_loss = traders_win_loss[traders_win_loss['loss'] != 0] 

In [49]:
traders_win_loss

,account_id,CP,win,loss,win_loss_ratio
299,1437622,JPI/JPY,75,1,75.0000
693,1562896,AUD/CAD,222,4,55.5000
1272,1651849,DJI/USD,51,1,51.0000
388,1463820,XAU/USD,87,2,43.5000
1276,1651849,EUR/CHF,35,1,35.0000
...,...,...,...,...,...
690,1562244,XAU/EUR,0,1,0.0000
689,1562244,XAG/USD,0,2,0.0000
688,1562244,USD/ZAR,0,1,0.0000
668,1562244,GAS/USD,0,3,0.0000


# Traders with best profit%

In [35]:
pd.options.display.float_format = '{:.4f}'.format

In [41]:
traders_profit_percent= final.groupby(['account_id']).agg({'invested': 'sum', 'profit_loss': 'sum', 'transactions':'sum'}).reset_index()

In [42]:
traders_profit_percent['profit%'] = traders_profit_percent['profit_loss'] / traders_profit_percent['invested']
traders_profit_percent= traders_profit_percent.sort_values(by = ['profit%'], ascending=False)
traders_profit_percent.head(5)

,account_id,invested,profit_loss,transactions,profit%
32,1574681,331129510.5200,35504595.5800,23388,0.1072
17,1474054,147666604862.7550,442496693.8650,603058,0.0030
96,1703339,355495525.4700,531931.9267,9600,0.0015
70,1664849,498811941.1900,386866.2400,17135,0.0008
80,1675145,829376836.4600,610100.2235,23938,0.0007


In [44]:
df = df[df['account_id'] == 1474054] 
df

,account_id,CP,oper_date,resulted_amount,cashflow,year,invested
886113,1474054,AUD/CAD,2020-06-10 05:14:16,50000.0000,-46865.5000,2020,46865.5000
886114,1474054,AUD/CAD,2020-06-10 05:14:17,300000.0000,-234315.0000,2020,234315.0000
886115,1474054,AUD/CAD,2020-06-10 07:20:55,250000.0000,46928.0000,2020,0.0000
886116,1474054,AUD/CAD,2020-06-10 07:20:56,0.0000,234665.0000,2020,0.0000
886117,1474054,AUD/CAD,2020-08-05 05:02:15,-1000.0000,955.9500,2020,0.0000
...,...,...,...,...,...,...,...
1489260,1474054,XAU/USD,2022-03-09 09:15:59,-155.0000,-199221.0000,2022,199221.0000
1489261,1474054,XAU/USD,2022-03-09 09:15:59,-55.0000,-199224.0000,2022,199224.0000
1489262,1474054,XAU/USD,2022-05-05 08:18:25,0.0000,-104577.8250,2022,104577.8250
1489263,1474054,XAU/USD,2022-07-05 09:08:34,-10.0000,17967.7000,2022,0.0000
